# 리랭커(Reranker)

<br>

<hr>

<br>

## Reranker
- Reranker(리랭커)는 현대적인 두 단계 검색 시스템(Two-Stage Retrieval System)에서 사용되는 핵심 컴포넌트
- Reranker는 검색 시스템의 두 번째 단계에서 작동하며, **초기 검색 결과의 정확도를 향상시키는 것을 목표**
  
  **Retriever가 대규모 문서 집합에서 관련성 있는 후보 문서들을 빠르게 추출한 후, Reranker는 이 후보 문서들을 더 정교하게 분석하여 최종적인 순위를 결정**

<br>

### 작동 원리
1. Retriever로부터 초기 검색 결과를 입력 받음
2. 쿼리와 각 후보 문서를 쌍으로 결합하여 처리
3. 복잡한 모델(주로 트랜스포머 기반)을 사용하여 각 쿼리-문서 쌍의 관련성을 평가
4. 평가 결과에 따라 문서들의 순위를 재조정
5. 최종적으로 재순위화된 결과를 출력

<br>

### 기술적 특징

<br>

#### 아키텍처
- 주로 BERT, RoBERTa 등의 트랜스포머 기반 모델 사용
- 교차 인코더(Cross-encoder) 구조 채택

<br>

#### 입력 형식
- 일반적으로 `[CLS] Query [SEP] Document [SEP]` 형태로 입력

<br>

#### 학습 방법
- 포인트와이즈(Pointwise): 개별 쿼리-문서 쌍의 관련성 점수 예측
- 페어와이즈(Pairwise): 두 문서 간의 상대적 관련성 비교
- 리스트와이즈(Listwise): 전체 순위 목록을 한 번에 최적화

<br>

<hr>

<br>

## Cross Encoder Reranker

<br>

#### 목적
- 검색된 문서들의 순위를 재조정하여 질문에 가장 관련성 높은 문서를 상위로 올림

<br>

#### 구조
- 질문과 문서를 동시에 입력으로 받아 처리

<br>

#### 작동 방식
- 질문과 문서를 하나의 입력으로 사용하여 유사도를 직접 출력
- Self-attention 메커니즘을 통해 질문과 문서를 동시에 분석

<br>

#### 장점
- 더 정확한 유사도 측정 가능
- 질문과 문서 사이의 의미론적 유사성을 깊이 탐색

<br>

#### 한계점
- 연산 비용이 높고 시간이 오래 걸림
- 대규모 문서 집합에 직접 적용하기 어려움

<br>

### 실제 사용
- 일반적으로 초기 검색에서 상위 `k`개의 문서에 대해서만 reranking 수행
- Bi-encoder로 빠르게 후보를 추출한 후, Cross encoder로 정확도를 높이는 방식으로 활용

<br>

#### 구현
- Hugging Face의 cross encoder 모델 또는 BAAI/bge-reranker와 같은 모델 사용
- LangChain 등의 프레임워크에서 `CrossEncoderReranker` 컴포넌트를 통해 쉽게 통합 가능

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
# 문서 로드
documents = TextLoader("./data/appendix-keywords.txt").load()

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 문서 분할
texts = text_splitter.split_documents(documents)

In [5]:
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)

In [6]:
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(
    search_kwargs={"k": 10}
)

In [7]:
query = "Word2Vec 에 대해서 알려줄래?"
docs = retriever.invoke(query)

pretty_print_docs(docs)


Document 1:

Open Source

정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업

Structured Data

정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이스, 스프레드시트 등에서 쉽게 검색하고 분석할 수 있습니다.
예시: 관계형 데이터베이스에 저장된 고객 정보 테이블은 구조화된 데이터의 예입니다.
연관키워드: 데이터베이스, 데이터 분석, 데이터 모델링

Parser
----------------------------------------------------------------------------------------------------
Document 2:

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.
예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.
연관키워드: 자연어 처리, 딥러닝, 텍스트 생성

FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Open Source
----------------------------------------------------------------------------------------------------
Document 3:

InstructGPT

정의: Ins

<br>

<hr>